# <font size = 5>Setting up</font>

In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import sys
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
import pandas as pd
from qutip import *
import scipy.constants
import pickle
from tkinter import filedialog # for prompting user to set analysis folder
import glob
import lyse
import h5py
from pathlib import PureWindowsPath, PurePosixPath
from tqdm import tqdm
from joblib import Parallel, delayed
import numba
from sympy.physics.wigner import wigner_3j, wigner_6j

analysis_directory = '/Users/huanbui/Documents/GitHub/labscript-suite/userlib/analysislib/common/analysis_scripts'
sys.path.append(analysis_directory)

image_utils_directory = '/Users/huanbui/Documents/GitHub/labscript-suite/userlib/analysislib/common/image_utils'
sys.path.append(image_utils_directory)

from analysis_utility_functions import *
from analysis_ROI_finding_functions import *
from analysis_functions import *

In [3]:
# suppress warnings: do at your own risk!
import warnings
warnings.filterwarnings("ignore")

In [5]:
# constants
ABSORPTION_LIMIT = 5.0
SPECIAL_CHARACTERS = "!@#$%^&*()-+?_=,<>/"

# physics constants
hbar = 1.05457182*10**(-34) # Js
c = 299792458 # m/s
h_planck = hbar*2*np.pi  # Js
me = 9.1093837015e-31    # electron mass
mn = 1.67493e-27         # neutron mass
eC = 1.60218e-19         # electric charge
muB = eC*hbar/(2*me)     # Bohr magneton
muB_cm1_G = muB / (100*c*h_planck*1e4) # Bohr magneton in units of cm-1 per Gauss
a0 = 5.2917720859e-11    # Bohr radius
muN = scipy.constants.physical_constants['nuclear magneton'][0]
eps0 = scipy.constants.epsilon_0
gS = 2.0023193043622
kB = 1.380649e-23 # Boltzmann constant
DebyeSI = 3.33564e-30

# Lithium constants
mLi6 = 9.9883414*10**(-27) # kg
Li6D2Gamma = 5.8724e6 # Hz Note that this is NOT angular freq
Li6D2lambda0 = 670.977338*10**(-9) # meters
Li6D2sigma0 = 3*(Li6D2lambda0)**2/(2*np.pi)
Li6Ahf_s12 = 152.1368407e6  # This is NOT angular freq
Li6Bhf_s12 = 0
Li6Ahf_p12 = 17.386e6 # Hz
Li6Bhf_p12 = 0
Li6Ahf_p32 = -1.155e6 # Hz
Li6Bhf_p32 = -0.10e6 # Hz
Li6I = 1
Li6gI = -0.0004476540

# Sodium-23 constants
mNa23 = 0.38175403519e-25 # kg
Na23D2Gamma = 9.794646e6 # Hz
Na23D2lambda0 = 589.158326415e-9 # meters
Na23D2sigma0 = 3*(Na23D2lambda0)**2/(2*np.pi)
Na23Ahf_s12 = 885.8130644050e6 # Hz
Na23Bhf_s12 = 0e6 # Hz
Na23Ahf_p12 = 94.4413e6 # Hz
Na23Bhf_p12 = 0e6 # Hz
Na23Ahf_p32 = 18.53415e6 # Hz
Na23Bhf_p32 = 2.72430e6 # Hz
Na23I = 3/2
Na23gI = -0.0008046108080

# Potassium-40 constants
mK40 = 39.9639984821*1.6605402e-27 # kg 
K40D2Gamma = 6.03511 # MHz
K40D2lambda0 = 766.700674872173e-9 # meters
K40D2sigma0 = 3*(K40D2lambda0)**2/(2*np.pi)
K40Ahf_s12 = -285.730824e6 # Hz
K40Bhf_s12 = 0
K40Ahf_p12 = -34.52325e6 # Hz
K40Bhf_p12 = 0
K40Ahf_p32 = -7.58510e6 # Hz
K40Bhf_p32 = -3.44590e6 # Hz
K40I = 4
K40gI = 0.00017649034

# NaK constants:
mK41 = 6.80187059497004e-26 # kg
mK40 = 6.63617749148248e-26 # kg
mK39 = 6.47007514485677e-26 # kg
u = 1.660538782e-27  # kg
Eh = 4.35974394e-18 
RKRcst = hbar/np.sqrt(2*u) * (10**10) * np.sqrt(1/(100 * c * hbar * 2*np.pi)) 
muNa39K = (mNa23 * mK39)/(mNa23 + mK39) * 1/u # atomic units
muNa40K = (mNa23 * mK40)/(mNa23 + mK40) * 1/u # atomic units 
muNa41K = (mNa23 * mK41)/(mNa23 + mK41) * 1/u # atomic units

# <font size = 5>Context</font>

In our experiment we begin with a thermal mixture of $^{23}$ Na in the hyperfine state $\ket{F=1, m_F=1}$ and $^{40}$ K in the hyperfine state $\ket{F=9/2, m_F = -9/2}$ at around 100 G. A Blackman pulse of duration ~ 100 ms with Rabi frequency ~ 5 kHz is turned on. The frequency $\omega_\text{rf}$ of the rf field is tuned across the resonance for the $m_F = -9/2 \to m_F = -7/2$ resonance in $^{40}$ K. There are two relevant Na-K scattering lengths which we denote $a_{-9/2}$ and $a_{-7/2}$. At 100 G, we have $a_{-9/2} = -1698 a_0$ and $a_{-7/2} = 644 a_0$, which means there is a two-body bound state of NaK with between $\ket{^{23}\text{Na}, F=1, m_F=1}$ and $\ket{^{40} \text{K}, F=9/2, m_F = -9/2}$ and no bound state of NaK with $^{40}$ K in $m_F = -9/2$. It is thus possible to create these bound states (a.k.a. Feshbach molecules) using the rf field when $\omega_\text{rf}$ is from the bare hyperfine resonance $\omega_\text{HF}$ an amount $E_b$ that is the molecule binding energy. The molecule association lineshape is known exactly. 

What we're after is the lineshape of the $^{40}$ K $m_F = -9/2 \to m_F = -7/2$ transition in the presence of Na. In our experiment, we observed the lineshape to have a threshold behavior with a high-frequency tail. We know this has to do with the interaction with the Na bath, have not found a satisfactory quantitative model for this rf response. The goal of this notebook is to put together what we know and what we need to know in order to find this lineshape. There are at least two ways to get the rf response for our system. One is via Fermi's Golden Rule, and the other is the Green's function-spectral function method. We shall focus on the latter.

# <font size = 5>Recipe for calculating rf response</font>

# <font size = 4, color=Blue>Momentum conservation</font>

Consider a single $\ket{^{23}\text{Na}, F=1, m_F=1}$ atom interacting with a $\ket{^{40} \text{K}, F=9/2, m_F = -9/2}$ atom. Assume that the Na atom has momentum $\hbar k_\text{Na}$ and the K atom has momentum $\hbar k_{K}$. We assume that the rf field with detuning $\delta$ from the hyperfine resonance does not change the momentum of the two-body system. This implies that the momenta of the particles are $\hbar (k_\text{Na} - q)$ and $\hbar (k_\text{K} + q)$, where $q$ is such that energy conservation is satisfied. 

# <font size = 4, color=Blue>Two-body scattering T-matrix</font>

We assume wo-body short-range contact interaction between the Na and K atoms. The bare interaction strength is given by 

$$ \frac{1}{g} = \frac{\mu}{4\pi a_s} - \sum_k^\Lambda  \frac{1}{2\epsilon_k} $$

where $\mu$ is the Na-K reduced mass and $\epsilon_k$ is the kinetic energy associated with the relative motion of the atoms. Just for clarity, we provide the reader with useful relations between the various momenta. These can be readily derived from the coordinate transformation, but it is nice to have them at our finger tips.

$$ P_\text{COM} = p_\text{Na} + p_\text{K} \quad\quad\quad P_\text{rel} = \mu \left( \frac{p_\text{Na}}{m_\text{Na}} - \frac{p_\text{K}}{m_\text{K}}  \right) $$

The two-body scattering $T$-matrix is often computed in the COM-rel coordinates. More explicitly, it depends on the relative momentum $\mathbf{k}$ of the two particles. As a reminder, the $T$-matrix is directly related to the scattering amplitude from the initial relative momentum  $\mathbf{k}$ to final relative momentum $\mathbf{k}'$ (see Sakurai's QM Eq. 7.2.19):

$$ f(\mathbf{k}',\mathbf{k}) = -\frac{1}{4\pi} \frac{2\mu}{\hbar^2} (2\pi)^3 \bra{\mathbf{k}' } T \ket{\mathbf{k}} $$

For the case of forward scattering, the amplitude is given by 

$$ f(\mathbf{k}'=\mathbf{k},\mathbf{k}) = f(\theta = 0, k) =  -\frac{1}{4\pi} \frac{2\mu}{\hbar^2} (2\pi)^3 T(k). $$

But of course we know that for low-energy, $s$-wave scattering, the (foward) scattering amplitude is given by (see, for example, Sakurai's QM Eq. 7.6.16)

$$ f_{\ell = 0}(k) = \frac{1}{k\cot \delta_{\ell = 0} - ik}  = \frac{a}{1 - ika}.$$


In the case of scattering between partiles of equal mass, the reduced mass is $\mu = m^2/2m = m/2$. The two-body $T$-matrix in the frequency domain is therefore (ignoring factors of $\hbar $ and $\pi$)

$$ T(\omega) =  \frac{4\pi}{m} \frac{1}{a^{-1} - \sqrt{- m\omega}},$$

where I have made the replacement $ik \to \sqrt{-m\omega}$. This is the expression found in e.g. [Meera's](https://arxiv.org/pdf/2308.06659v3.pdf) and [Leyronas's](https://arxiv.org/abs/1506.07334).

For our case, the $T$-matrix should be 

$$ T(\omega) =  \frac{4\pi}{2\mu} \frac{1}{a^{-1} - \sqrt{- 2\mu\omega}}.$$

Reinstating $\hbar$, we find that the $T$-matrix has a pole at $\omega = \hbar^2/2ma^2$ whenever $a > 0$. This is precisely the statement that the two-body problem admits a bound state when the interaction is repulsive. 

# <font size = 4, color=Blue>Self-energy in the virial expansion for interacting fermions</font>

Following [Leyronas](https://arxiv.org/abs/1506.07334) and [Leyronas](https://arxiv.org/pdf/1109.4794.pdf), we begin with the free fermion propagator, which in imaginary time is given by 
$$ G^{(0)}(\mathbf{k}, \tau) = e^{-(\epsilon_\mathbf{k} - \mu)\tau} [ n_F(\epsilon_\mathbf{k} - \mu) - \Theta(\tau)],$$

where $\mu$ is the chemical potential, $\epsilon_\mathbf{k} = \hbar^2 k^2 / 2m_F$, $n_F(x) = 1/(e^{\beta x} + 1)$ is the Fermi-Dirac distribution, $\Theta(x)$ is the Heaviside step function, and $\tau$ is the imaginary time. The derivation for the free-fermion propagator can be found in many textbooks/lecture notes (e.g., [this page](https://webspace.science.uu.nl/~morai101/oldWebsite/public_html/SFT06four.pdf)), but I can reproduce it here:

A free particle of mass $m$ has the Hamiltonian
$$
H = \sum_\mathbf{k} \epsilon_\mathbf{k} a^\dagger_\mathbf{k} a_\mathbf{k}
$$
with $\epsilon_\mathbf{k} = \hbar^2 k^2 / 2m$. Time independence comes in via $ a_\mathbf{k}(t) = e^{-i\epsilon_\mathbf{k} t / \hbar} a_\mathbf{k}(0) = e^{-i\epsilon_\mathbf{k} t / \hbar} a_\mathbf{k}$. The Green's function for the free particle is given by 
$$
G(\mathbf{k}, t) = -\langle \mathcal{T}[a_\mathbf{k}(t) a_\mathbf{k}^\dagger] \rangle
$$

The interpretation is simple: it is the amplitude of annihilating a particle of moomentum $\hbar \mathbf{k}$ at $t'=0$ and creating a particle of momentum $\hbar \mathbf{k}$ at $t'=t$. Here, $\mathcal{T}[\cdot]$ denotes the time-ordering operator. For fermions, we have $\langle a_\mathbf{k}^\dagger a_\mathbf{k} \rangle = n_F(\mathbf{k})$ and $\mathcal{T}[\cdot] \to \mathcal{T}_F[\cdot]$ is the fermionic time-ordering operator. By applying the fermionic anti-commutation rule and plugging this into the definition for the Green's function, we find the result above after a little bit of simplification plus going to imaginary time.

In the high-temperature limit, the Fermi-Dirac distrbution can be expanded in powers of the fugacity $z = e^{\beta \mu} \ll 1$ as:
$$ n_F(\epsilon_\mathbf{k}, \mu) = \sum_{n\geq 1} (-1)^{n+1} z^n e^{-n\beta \epsilon_\mathbf{k}}.$$

The free fermion Green's function is then 
$$ G^{(0)}(\mathbf{k}, \tau) = e^{\mu \tau} \left[ \sum_{n\geq 0} G^{(0,n)} (\mathbf{k}, \tau) z^n \right], $$

where 
$$ 
\begin{align*}
G^{(0,0)} (\mathbf{k}, \tau) &= -\Theta(\tau)e^{-\epsilon_\mathbf{k}\tau} \\
G^{(0,n)} (\mathbf{k}, \tau) &= (-1)^{n-1} e^{-\epsilon_\mathbf{k}\tau} e^{-n\beta \epsilon_\mathbf{k}}, \quad n\geq 1.
\end{align*} 
$$

We see that $G^{(0,0)}$ is retarded, but not $G^{(0,n)}$ for $n\geq 1$. In the language of Feynman diagrams, $G^{(0,0)}$ is a  line going from left to right in the direction of increasing time, while $G^{(0,n)}$ can go either way (see [Hofmann](https://arxiv.org/pdf/1309.6573.pdf) for an example of such a Feynman diagram). The leading order in $z$ is thus given by the diagram with the least number of advanced propagators. In this case it would be $G^{(0,1)}$.

The self-energy operator $\Sigma$ relates the free and dressed fermion Green's functions via 
$$ G = G^{(0)} + G^{(0)} \Sigma G \implies \Sigma = [G^{(0)}]^{-1} - [G]^{-1}.$$

The contribution to the self energy with the lowest order in $z$ is $G^{(0,1)}$. 

For the case interacting fermions of spins $\uparrow, \downarrow$ of equal mass, [Meera](https://arxiv.org/pdf/1309.0272.pdf) shows that the self-energy, to lowest order of $z$, has the form of an integral over some degrees of freedom of the $T$-matrix multiplied by $G^{(0,1)}$. The result is Eq. (5) of [Leyronas](https://arxiv.org/abs/1506.07334):
$$
\Sigma^{(1)}(\mathbf{k},\tau) = z \int \frac{d^3P}{(2\pi)^3} e^{\mu\tau} e^{-(\beta-\tau) \epsilon_\mathbf{P-k}} T(\mathbf{P},\tau).
$$

This formula seems to be "out of the blue" for the unfamiliar reader, but [Meera](https://arxiv.org/pdf/1309.0272.pdf) provides a Feynman diagram which better elucidates its origins. Basically, we want to think of the contribution to the self-energy to lowest order of $z$ is a "product" of the $T$-matrix and the Green's function of the some particle which our particle is interacting with, summed over all the possible center-of-mass momenta of the two particles ([Leyronas](https://arxiv.org/pdf/1109.4794.pdf)). So, in the case of two interacting fermions of the equal mass, the self-energy has the form
$$ 
\Sigma^{(1)}_\uparrow(\mathbf{k}, \tau) = \int_P d\mathbf{P} \, G_\downarrow(\mathbf{P} - \mathbf{k},\tau) T(\mathbf{P}, \tau).
$$

How does the $T$-matrix $T(\mathbf{P},\tau)$ in the imaginary time domain relate to the $T$-matrix $T(\omega)$ in the frequency domain? It turns out that due to Galilean invariance ($\textcolor{red}{\text{why?}}$), we have $T(\mathbf{P},\tau) = e^{-\tau P^2/4m} T(0,\tau)$. The quantity $T(0,\tau)$ is then directly related to $T(\omega)$ via the Laplace transform. From [Leyronas](https://arxiv.org/abs/1506.07334), we have
$$ 
T(\mathbf{P},\tau) = e^{-\frac{\tau P^2}{4m}} \int_{C_\gamma} \frac{d\omega}{2\pi i} e^{-\tau \omega} T(\omega).
$$

This integral is "simply" the inverse Laplace transform. The contour $C_\gamma$ is called the Bromwich contour. We won't go into the details here, but since $T(\omega)$ is already known, we can compute this integral. The result is 
$$
T(\mathbf{P}, \tau) = e^{-\frac{\tau P^2}{4m}} \left[ \Theta(a^{-1}) Z_m e^{E_b \tau} + \int_0^\infty e^{-\tau x} \rho(x)\,dx\right]
$$

The integral is evaluated by deforming the contour along the real axis. Here $Z_m  = 8\pi / m^2 a$ is the molecular residue, and $\rho(x) = -1/\pi \,\text{Im}[T(x + i0^+)]$ is the spectral density of the $T$-matrix. Leyronas inteprets this as follows: the first term comes from the two-body bound state (which exists in the case where $a>0$), while the second term comes from the two-particle scattering states continuum of energy $x$. Putting this expression back into the expression for the self-energy, we find to first order in $z$ after Fourier-transforming and analytically-continuing: 
$$
\Sigma^{(1)}(\mathbf{k}, \omega) = z F(\mathbf{k}, \omega + \mu + i0^+),
$$

where
$$
F(\mathbf{k}, E) = \int \frac{d^3P}{(2\pi)^3} e^{-\beta  \frac{(\mathbf{P}- \mathbf{k})^2}{2m}}
\left\{
\int_0^\infty \frac{}{}\,dx \frac{\rho(x)}{E - \left[ \frac{P^2}{4m} + x - \frac{(\mathbf{P} - \mathbf{k})^2}{2m} \right]}
+
\Theta(a^{-1})\frac{Z_m}{E - \left[ \frac{\mathbf{P}^2}{4m} - E_b - \frac{(\mathbf{P} - \mathbf{k})^2}{2m}\right]}
\right\}.
$$

The case of interacting fermions ($\uparrow, \downarrow$) of equal masses is then done, at least to first order in $z$. From the self-energy, we can then compute the retarded Green's function via the Dyson's equation. The spectral function is directly related to the imaginary part of the Green's function. Following [Zwerger](https://arxiv.org/pdf/0904.1333.pdf), one can then extract the rf response from the spectral function depending on the initial and final states.

What we're after, however, is the solution for the case where we have a fermion interacting with a boson with a different mass. 

# <font size = 4, color=Blue>Self-energy a fermion interacting with a boson of unequal mass</font>

We would like to treat a slightly different scenario, with two main differences: the interacting particles have different quantum statistics and unequal mass. The self-enegrgy for the fermion is now different: it is now the convolution of the $T$-matrix with the boson Green's function. 

$\textcolor{red}{\text{What is the bosonic Green's function in the time domain? How does it looks like in the virial expansion?}}$

Without knowing the exact form of the bosonic Green's function, what we can say is that the fermionic self-energy is now different from the last section for two reasons: (1) the two-body system consists of two particles with different quantum statistics and (2) the particles have unequal mass. In any case, the fermionic self-energy should look like
$$
\Sigma_F(\mathbf{k}, \omega) \sim \sum_? T \times G_B
$$
where $T$ denotes the two-body $T$-matrix and $G_B$ is the bosonic Green's function.

To make the formalism consistent with what was done in the previous section, we should also find a virial expansion for this and obtain the self-energy to lowest order of the fugacity $z$.

Useful references:
1/ [Ohashi](https://arxiv.org/pdf/1705.02591.pdf)
2/ [Ohashi](https://arxiv.org/pdf/1907.12257.pdf)

We now attempt to obtain the free boson Green's function in the virial expansion. First, we expand the Bose-Einstein distribution in powers of the fugacity $z_B = e^{\beta \mu_B}$:
$$
n_B(\mathbf{k},\mu) = \frac{1}{e^{\beta(\epsilon_\mathbf{k} - \mu_B)} - 1} = \sum_{n\geq 1} z^n e^{-n \beta \epsilon_\mathbf{k}}.
$$

Next, we find the free boson Green's function from the steps outlined in the beginning of the previous section:
$$
\begin{align*}
G_B^{(0)}(\mathbf{k}, \tau) 
&=  -e^{-(\epsilon_\mathbf{k} - \mu_B)\tau} 
\left\{
    [1 + n_B(\epsilon_\mathbf{k} - \mu_B) ] \Theta(\tau) +  n_B(\epsilon_\mathbf{k} - \mu_B) \Theta(-\tau)
\right\} \\
&= -e^{-(\epsilon_\mathbf{k} - \mu_B) \tau} [n_B(\epsilon_\mathbf{k} - \mu_B)  + \Theta(\tau) ]
\end{align*}.
$$

In powers of the fugacity $\mu_B$, we have
$$
G^{(0)}_B(\mathbf{k}, \tau) = e^{\mu_B \tau} \sum_{n\geq 0} G_B^{(0,n)} (\mathbf{k}, \tau) z^n
$$

where
$$ 
\begin{align*}
G_B^{(0,0)} (\mathbf{k}, \tau) &= -\Theta(\tau)e^{-\epsilon_\mathbf{k}\tau} \\
G_B^{(0,n)} (\mathbf{k}, \tau) &= e^{-\epsilon_\mathbf{k}\tau} e^{-n\beta \epsilon_\mathbf{k}}, \quad n\geq 1.
\end{align*} 
$$

